In [1]:
import torch
device = "mps" if torch.backends.mps.is_available() else "cpu"

In [2]:
device

'mps'

In [3]:
import torch
import torch.nn as nn
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset

/Users/snehsuresh/anaconda3/envs/mynewenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
teacher_model_name = "distilbert-base-uncased"
teacher_model = AutoModelForSequenceClassification.from_pretrained(
    teacher_model_name, num_labels=2
)
teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/snehsuresh/anaconda3/envs/mynewenv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
dataset = load_dataset("imdb")

def tokenize_function(examples):
    return teacher_tokenizer(examples["text"], truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 25000/25000 [00:05<00:00, 4286.41 examples/s]


In [6]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [10]:
import torch
import psutil


def get_hardware_info():
    # Check if a CUDA GPU is available
    gpu_available = torch.cuda.is_available()

    # Check if MPS is available (for Apple Silicon devices like M1/M2)
    mps_available = torch.backends.mps.is_available()

    # Get available system RAM
    total_ram = psutil.virtual_memory().total / (1024**3)  # Convert to GB
    available_ram = psutil.virtual_memory().available / (1024**3)  # Convert to GB

    # Get CPU core count
    cpu_cores = psutil.cpu_count(logical=False)  # Physical cores
    logical_cores = psutil.cpu_count(logical=True)  # Logical cores

    # Get GPU details if a CUDA GPU is available
    if gpu_available:
        gpu_count = torch.cuda.device_count()
        gpu_mem = torch.cuda.get_device_properties(0).total_memory / (
            1024**3
        )  # Convert to GB
        gpu_type = "CUDA"

    # Get MPS details if an MPS GPU is available
    elif mps_available:
        gpu_count = 1  # MPS typically means 1 Apple GPU
        gpu_mem = None  # No easy way to fetch memory for MPS currently
        gpu_type = "MPS"

    # If neither CUDA nor MPS is available
    else:
        gpu_count = 0
        gpu_mem = 0
        gpu_type = "None"

    # Combine the hardware info into a dictionary
    hardware_info = {
        "gpu_available": gpu_available,
        "mps_available": mps_available,
        "gpu_type": gpu_type,
        "gpu_count": gpu_count,
        "gpu_memory": gpu_mem,
        "total_ram": total_ram,
        "available_ram": available_ram,
        "cpu_cores": cpu_cores,
        "logical_cores": logical_cores,
    }

    return hardware_info


# Check hardware info
hardware_info = get_hardware_info()
print(hardware_info)

{'gpu_available': False, 'mps_available': True, 'gpu_type': 'MPS', 'gpu_count': 1, 'gpu_memory': None, 'total_ram': 8.0, 'available_ram': 1.30828857421875, 'cpu_cores': 8, 'logical_cores': 8}


In [11]:
class AdaptiveStudentModel(nn.Module):
    def __init__(self, teacher_model_name, hardware_info):
        super(AdaptiveStudentModel, self).__init__()

        # Load teacher model to get its hidden size
        self.teacher_model = AutoModelForSequenceClassification.from_pretrained(
            teacher_model_name, num_labels=2
        )
        hidden_size = self.teacher_model.config.hidden_size

        # Adjust the student model architecture based on hardware
        if hardware_info["gpu_available"]:
            if hardware_info["gpu_memory"] < 4:  # For low-end GPUs
                self.student_model = nn.Linear(hidden_size, 128)  # Fewer units
            else:
                self.student_model = nn.Linear(
                    hidden_size, 256
                )  # Higher-end GPUs get a larger model
        elif hardware_info["mps_available"]:
            self.student_model = nn.Linear(
                hidden_size, 128
            )  # Adapt for MPS (Apple Silicon)
        else:
            self.student_model = nn.Linear(
                hidden_size, 64
            )  # Smaller model for CPU-only devices

        self.output_layer = nn.Linear(
            self.student_model.out_features, 2
        )  # Output layer size stays the same

    def forward(self, input_ids, attention_mask):
        teacher_outputs = self.teacher_model(input_ids, attention_mask=attention_mask)
        student_outputs = self.student_model(teacher_outputs[1])  # Use pooled output
        return self.output_layer(student_outputs)

In [ ]:
class AdaptiveStudentModel(nn.Module):
    def __init__(self, teacher_model_name, hardware_info):
        super(AdaptiveStudentModel, self).__init__()
        
        # Load teacher model to get its hidden size
        self.teacher_model = AutoModelForSequenceClassification.from_pretrained(teacher_model_name, num_labels=2)
        hidden_size = self.teacher_model.config.hidden_size
        
        # Adjust the student model architecture based on hardware
        if hardware_info["gpu_available"]:
            if hardware_info["gpu_memory"] < 4:  # For low-end GPUs
                self.student_model = nn.Linear(hidden_size, 128)  # Fewer units
            else:
                self.student_model = nn.Linear(hidden_size, 256)  # Higher-end GPUs get a larger model
        elif hardware_info["mps_available"]:
            self.student_model = nn.Linear(hidden_size, 128)  # Adapt for MPS (Apple Silicon)
        else:
            self.student_model = nn.Linear(hidden_size, 64)  # Smaller model for CPU-only devices

        self.output_layer = nn.Linear(self.student_model.out_features, 2)  # Output layer size stays the same

    def forward(self, input_ids, attention_mask):
        teacher_outputs = self.teacher_model(input_ids, attention_mask=attention_mask)
        student_outputs = self.student_model(teacher_outputs[1])  # Use pooled output
        return self.output_layer(student_outputs)

In [12]:
# Distillation loss function
def distillation_loss(student_logits, teacher_logits, temperature=2.0):
    teacher_probs = nn.functional.softmax(teacher_logits / temperature, dim=-1)
    student_probs = nn.functional.log_softmax(student_logits / temperature, dim=-1)
    return nn.KLDivLoss()(student_probs, teacher_probs) * (temperature**2)